## Скрипт для создания заявок Subzero на несколько аккаунтов МС


In [2]:
import pandas as pd
import requests
import json
from threading import Thread

In [4]:
namespace = 4  # Неймспейс
ticket_number = 2  # Количество заявок на 1 аккаунт
thread_number = 20  # Количество потоков

url = f'https://subzero-billing-{namespace}.testms-test.lognex.ru/api/clinton/1.0/ticket'
headers = {'Content-Type': 'application/json'}

In [30]:
#column = ['id']
accounts = pd.read_csv('lognex_billing_billingaccount.csv',header=None)
accounts=accounts.values.tolist()

In [32]:
def bomber_to_many():
    counter = 0
    with open('json3.json', 'r') as json3:
        sub_data = json.load(json3)
    for account in accounts:
        sub_data['accountId'] = (''.join(account))
        for i in range(ticket_number):
            counter += 1
            response = requests.post(url, data=json.dumps(sub_data), headers=headers)
            print(f'Заявка для {account} создана')
            #return response
    print('------------------------------------------------')
    print(f'итого, создано заявок: {counter}')

In [ ]:
bomber_to_many()

In [5]:

counter=0

with open('json3.json', 'r') as json3:
    sub_data = json.load(json3)
    for account in accounts:
        sub_data['accountId'] = (''.join(account))
        with open('json3.json', 'w') as json3:
            json3.write(json.dumps(sub_data,))

        for i in range(2):
            counter +=1
            response = requests.post(url, data=json.dumps(sub_data), headers=headers)
            print(f'Заявка для {account} создана')
    print('------------------------------------------------')
    print(f'итого, создано заявок: {counter}')

Заявка для ['4615c8f6-0e7b-11e2-06e0-3c4a92f3a0a7'] создана
Заявка для ['4615c8f6-0e7b-11e2-06e0-3c4a92f3a0a7'] создана
Заявка для ['0f75fc77-63a7-11ed-0a83-02fa0000728a'] создана
Заявка для ['0f75fc77-63a7-11ed-0a83-02fa0000728a'] создана
Заявка для ['9a0b3d33-62cf-11ed-0a83-02fa00000002'] создана
Заявка для ['9a0b3d33-62cf-11ed-0a83-02fa00000002'] создана
Заявка для ['100eea6a-63a7-11ed-0a83-02fa00007293'] создана
Заявка для ['100eea6a-63a7-11ed-0a83-02fa00007293'] создана
Заявка для ['ab923454-62cf-11ed-0a83-02fa0000000b'] создана
Заявка для ['ab923454-62cf-11ed-0a83-02fa0000000b'] создана
Заявка для ['c42d92e1-6396-11ed-0a83-02fa00007210'] создана
Заявка для ['c42d92e1-6396-11ed-0a83-02fa00007210'] создана
Заявка для ['8f8baf75-63b3-11ed-0a83-02fa000072a6'] создана
Заявка для ['8f8baf75-63b3-11ed-0a83-02fa000072a6'] создана
Заявка для ['90505033-63b3-11ed-0a83-02fa000072af'] создана
Заявка для ['90505033-63b3-11ed-0a83-02fa000072af'] создана
Заявка для ['90ec3b46-63b3-11ed-0a83-02f

## Потоки

In [40]:
# Потоки

thread1 = Thread(target=bomber_to_many)
thread2 = Thread(target=bomber_to_many)
thread3 = Thread(target=bomber_to_many)
thread4 = Thread(target=bomber_to_many)
thread5 = Thread(target=bomber_to_many)
thread6 = Thread(target=bomber_to_many)
thread7 = Thread(target=bomber_to_many)
thread8 = Thread(target=bomber_to_many)
thread9 = Thread(target=bomber_to_many)
thread10 = Thread(target=bomber_to_many)
thread11 = Thread(target=bomber_to_many)
thread12 = Thread(target=bomber_to_many)
thread13 = Thread(target=bomber_to_many)
thread14 = Thread(target=bomber_to_many)
thread15 = Thread(target=bomber_to_many)
thread16 = Thread(target=bomber_to_many)
thread17 = Thread(target=bomber_to_many)
thread18 = Thread(target=bomber_to_many)
thread19 = Thread(target=bomber_to_many)
thread20 = Thread(target=bomber_to_many)


def start_threads():
    thread1.start()
    thread2.start()
    thread3.start()
    thread4.start()
    thread5.start()
    thread6.start()
    thread7.start()
    thread8.start()
    thread9.start()
    thread10.start()
    thread11.start()
    thread12.start()
    thread13.start()
    thread14.start()
    thread15.start()
    thread16.start()
    thread17.start()
    thread18.start()
    thread19.start()
    thread20.start()


def stop_threads():
    thread1.join()
    thread2.join()
    thread3.join()
    thread4.join()
    thread5.join()
    thread6.join()
    thread7.join()
    thread8.join()
    thread9.join()
    thread10.join()
    thread11.join()
    thread12.join()
    thread13.join()
    thread14.join()
    thread15.join()
    thread16.join()
    thread17.join()
    thread18.join()
    thread19.join()
    thread20.join()

In [ ]:
%%time
start_threads()
stop_threads()